### Import libraries

In [6]:
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
import pyalex
import pandas as pd
import networkx as nx
import numpy as np
import typing as tp
from tqdm import tqdm
import matplotlib.pyplot as plt
import datetime as dt
import os
pyalex.config.email = "v.veselov1@g.nsu.ru"
import warnings
warnings.filterwarnings("ignore")

In [7]:
data = pd.read_parquet('data_to_pqt.pqt')

In [8]:
articles_id = data['Article OpenAlex ID'].apply(lambda x: x.split('/')[-1]) # получаем id статей

In [9]:
authors_id = data['Author OpenAlex ID'].apply(lambda x: x.split('/')[-1]).unique() # получаем id авторов

In [10]:
def parse_authors_openalex(start_it: int=0) -> tp.Dict:
    """Парсинг авторов с openalex.

    Args:
        start_it (int): стартовая итерация для парсинга.

    Returns:
        save_dict_authors (Dict[int, Dict]): сохраненные данные по авторам.
        
    """
    save_dict_authors = {}
    for it, id in tqdm(enumerate(authors_id), total=len(authors_id)):
        if it < start_it:
            continue
        if id not in save_dict_authors:
            author = Authors()[id]
            try:
                save_dict_authors[id] = author
            except:
                print(f'exception: author {id} not found!')
                continue
                
        if it % 100 == 0 and it != start_it:
            pd.to_pickle(save_dict_authors, f'save_dict_authors_{it}.pkl')
    return save_dict_authors


In [11]:
def parse_papers_openalex() -> tp.Dict:
    """Парсинг статей с openalex.

    Args:
        start_it (int): стартовая итерация для парсинга.

    Returns:
        save_dict_papers (Dict[int, Dict]): сохраненные данные по статьям.
        
    """
    save_dict_papers = {}
    for it, id in tqdm(enumerate(articles_id.unique()), total=len(articles_id.unique())):
        if it < start_it:
            continue
        if id not in save_dict_papers:
            paper = Works()[id]
            try:
                save_dict_papers[id] = paper
            except:
                print(f'exception: paper {id} not found!')
                continue
                
        if it % 3000 == 0 and it != 0:
            pd.to_pickle(save_dict_papers, f'save_dict_papers_{it}.pkl')
    pd.to_pickle(save_dict_papers, f'save_dict_papers.pkl')
    return save_dict_papers


In [12]:
# читаем спаршенные данные 

saved_papers = pd.read_pickle('save_dict_papers_70000.pkl')

In [13]:
def create_paper_authors_rel(saved_papers: tp.Dict) -> tp.Dict:
    """Выделение основных доменов данных.

    Args:
        saved_papers (Dict[int, Dict]): сохраненные данные по статьям.

    Returns:
        paper_authors (Dict[int, Dict[str, Optional]]): словарь основных доменов данных.

    """
    paper_authors = {}
    k = 0
    for id in tqdm(saved_papers):
        authorships = saved_papers[id]['authorships']
        authors = []
        aff = []
        paper_authors[id] = {}
        try:
            for auth in authorships:
                author = auth['author']['id'].split('/')[-1]
                af = auth['raw_affiliation_strings']
                authors.append(author)
                aff.extend(af)
            paper_authors[id]['title'] = saved_papers[id]['title']
            paper_authors[id]['authors'] = authors
            paper_authors[id]['authors_affiliations'] = aff
            paper_authors[id]['topics'] = [(i['display_name'], i['score'])  for i in saved_papers[id]['topics'] if 'display_name' in i] 
            paper_authors[id]['keywords'] = [(i['display_name'], i['score'])  for i in saved_papers[id]['keywords'] if 'display_name' in i]  
            if saved_papers[id]['primary_topic'] is not None:
                paper_authors[id]['prim_topic'] = saved_papers[id]['primary_topic']['display_name']
                paper_authors[id]['subfield'] = saved_papers[id]['primary_topic']['subfield']['display_name']
                paper_authors[id]['field'] =  saved_papers[id]['primary_topic']['field']['display_name']
                paper_authors[id]['domain'] = saved_papers[id]['primary_topic']['domain']['display_name']
            else:
                k+=1
        except:
            print(id)
            continue
        
        paper_authors[id]['pub_date'] = dt.datetime.strptime(f"{(saved_papers[id]['publication_date'])}",  "%Y-%m-%d")
        
    return paper_authors

In [14]:
paper_authors = create_paper_authors_rel(saved_papers)

  2%|▋                                   | 1352/77002 [00:00<00:33, 2270.47it/s]

W3010671326
W2887469692
W2918172693


  6%|██▎                                 | 4845/77002 [00:01<00:15, 4621.60it/s]

W2807156318
W4226308094


100%|██████████████████████████████████| 77002/77002 [00:05<00:00, 15100.17it/s]


In [15]:
def update_exist_table(data: pd.DataFrame) -> pd.DataFrame:
    """Дополняем существующие данные новыми доменами.

    Args:
        data (DataFrame): исходные данные без доменов.
        
    Outputs:
        data (DataFrame): данные c доменами.
    """
    data['topics'] = None
    data['keywords'] = None
    for i, row in tqdm(data.iterrows(), total=data.shape[0]):
        try:
            paper_id = row['Article OpenAlex ID'].split('/')[-1]
            if paper_id in paper_authors:
                data.at[i, 'pub_date'] = paper_authors[paper_id]['pub_date']
                data.at[i, 'keywords'] = [i[0] for i in paper_authors[paper_id]['keywords']]
                data.at[i, 'topics'] = [i[0] for i in paper_authors[paper_id]['topics']]
                try: 
                    data.at[i,'prim_topic'] = paper_authors[paper_id]['prim_topic']
                    data.at[i,'subfield'] = paper_authors[paper_id]['subfield']
                    data.at[i,'field'] =  paper_authors[paper_id]['field']
                    data.at[i,'domain'] = paper_authors[paper_id]['domain']
                except:
                    continue
        except:
            print(i)
            continue
    return data

In [16]:
data = data[data['Author OpenAlex ID'] != 'https://openalex.org/A5005891681'].reset_index(drop=True)

In [17]:
data = update_exist_table(data)

  1%|▎                                   | 1540/160755 [00:02<04:14, 626.77it/s]

1413
1424
1439


  3%|▉                                   | 4150/160755 [00:07<03:58, 655.85it/s]

4070
4085


  4%|█▍                                  | 6377/160755 [00:10<04:34, 562.90it/s]

6238


  4%|█▌                                  | 6752/160755 [00:11<04:04, 630.13it/s]

6635


100%|██████████████████████████████████| 160755/160755 [03:07<00:00, 857.59it/s]


In [18]:
# задаем времена начала и конца периода, в котором будем рассматривать матрицу взаимодействий C_ij

t_start, t_end = dt.datetime.strptime("2012-02-01", "%Y-%m-%d"), dt.datetime.strptime("2023-06-06", "%Y-%m-%d")

In [263]:
# удаляем пропуски

data_notna = data[~data.topics.isna()]

In [705]:
# example_author = 'Vitali M. Volosi'

In [706]:
# shuffled_names = data_notna[data_notna['Author name'] == example_author]['Author name'].sample(frac=1)
# for i in shuffled_names.index:
#     shuffled_names[i] = shuffled_names[i] + "_1" if i % 2 != 0 else shuffled_names[i] + "_2"

In [707]:
# data_notna.loc[data_notna['Author name'] == example_author, 'Author name'] = shuffled_names

In [708]:
# data_notna.loc[data_notna['Author name'] == example_author + "_1", 'Author OpenAlex ID'] = data_notna.loc[data_notna['Author name'] == example_author + "_1", 'Author OpenAlex ID'] + '_'

In [380]:
# получаем id всех авторов из таблицы

all_authors = list(np.unique(data_notna['Author OpenAlex ID']))

In [381]:
# получаем отношение между статьями, авторами, временем публикации 

authors_rel = data_notna.groupby(['Article OpenAlex ID', 'pub_date'])['Author OpenAlex ID'].agg(list)

In [382]:
# текущее время
t_current = dt.datetime.now()

# перевод даты в месяца, года
in_months = lambda x: x.year * 12 + x.month
in_years = lambda x: x.year

# подсчет разницы между датами в месяцах, годах
to_month = lambda a,b: b.month - a.month + (b.year - a.year)*12
to_year = lambda a, b: b.year - a.year

In [383]:
def coop_strength_matrix(t_start: dt.datetime, t_end: dt.datetime, time_func_in=in_months) -> tp.Tuple[np.array, np.array]:
    """Создаем матрицу коопераций с_ij.
    
    Args:
        t_start (Datetime): время начала взаимодействий.
        t_end (Datetime): время конца взаимодействий.
        time_func (Callable[Datetime, int]): перевод времени в года или месяцы.

    Returns:
        first_communicate, authors_matrix * mask_matrix (Tuple[Array, Array]): матрицы первых взаимодействий авторов в годах/месяцах и силы коопераций между авторами. 
    """
    authors_matrix = np.zeros((len(all_authors), len(all_authors)))
    mask_matrix = np.zeros((len(all_authors), len(all_authors)))
    first_communicate = np.ones((len(all_authors), len(all_authors))) * 1e6
    for it, aus in tqdm(enumerate(authors_rel.values), total=len(authors_rel.values)):
        for a in aus:
            for b in aus:
                if t_start <= authors_rel.index[it][1] <= t_end: 
                    f_com = first_communicate[all_authors.index(a), all_authors.index(b)]
                    if  f_com > time_func_in(authors_rel.index[it][1]):
                        first_communicate[all_authors.index(a), all_authors.index(b)] = time_func_in(authors_rel.index[it][1])
    # print(first_communicate)
    for it, aus in tqdm(enumerate(authors_rel.values), total=len(authors_rel.values)):
        for a in aus:
            for b in aus:
                if a == b:
                    mask_matrix[all_authors.index(a), all_authors.index(b)] = 1
                if t_start <= authors_rel.index[it][1] <= t_end:
                    if a == b: continue
                    f_com = first_communicate[all_authors.index(a), all_authors.index(b)]
                    authors_matrix[all_authors.index(a), all_authors.index(b)] += (time_func_in(authors_rel.index[it][1]) - f_com) / (time_func_in(t_current) - f_com) if f_com != 1e-6 else 0  
                    mask_matrix[all_authors.index(a), all_authors.index(b)] = 1
    # print(mask_matrix)
    print(mask_matrix.sum(axis=1).max())
    authors_matrix = (lambda x: 1 / (1 + np.e**(-x)))(authors_matrix)
    authors_matrix += np.eye(len(authors_matrix)) / 2
    
                    
                
    return  first_communicate, authors_matrix * mask_matrix

In [384]:
first_coop, matrix_coop = coop_strength_matrix(t_start, t_end, time_func_in=in_years)

100%|███████████████████████████████████| 76997/76997 [00:56<00:00, 1357.10it/s]


175.0


In [385]:
id2author = dict(zip(data_notna['Author OpenAlex ID'], data_notna['Author name']))

In [386]:
authors_ = list(map(lambda x: id2author[x], all_authors))

In [387]:
# Cоздаем отношение автор - вектор признаков (b_i = (b1,..., b6) ). 

author_vectors = data_notna.groupby('Author name').agg({'keywords': lambda x: list(set(sum(x, []))),
                                                        'topics': lambda x: list(set(sum(x, []))),
                                                        'prim_topic': lambda x: list(set(x)),
                                                        'subfield': lambda x: list(set(x)),
                                                        'field': lambda x: list(set(x)),
                                                        'domain': lambda x: list(set(x))
                                                       })

In [388]:
def create_pairwise_score_matrix(author_vectors: pd.DataFrame, authors: tp.List) -> pd.DataFrame:
    """Создаем матрицу сходства по атрибутам S_ij.

    Args:
        author_vectors (DataFrame): авторы и их атрибуты.
        authors (List): все авторы.

    Returns:
        
    """
    authors_pairwise_score_matrix = pd.DataFrame(np.zeros((len(authors), len(authors))), columns=authors, index=authors)
    for a_name_1, a_vec_1 in tqdm(author_vectors.iterrows(), total=author_vectors.shape[0]):
        for a_name_2, a_vec_2 in author_vectors.iterrows(): 
            pairwise_score = 0
            for feature_1, feature_2 in zip(a_vec_1, a_vec_2):
                cnt_features = max(len(feature_1), len(feature_2))
                cnt_eq_features = len(set(feature_1) & set(feature_2))
                feature_score = cnt_eq_features / cnt_features if cnt_features > 0 else 0
                pairwise_score += feature_score
            authors_pairwise_score_matrix.loc[a_name_1, a_name_2] = pairwise_score / 6
    return authors_pairwise_score_matrix

In [389]:
authors_pairwise_score_matrix = create_pairwise_score_matrix(author_vectors, authors_)

100%|███████████████████████████████████████| 6959/6959 [38:51<00:00,  2.98it/s]


In [ ]:
# authors_pairwise_score_matrix.to_csv('authors_pairwise_score_matrix.csv')

In [ ]:
# authors_pairwise_score_matrix.to_csv('matrix_S.csv')

In [274]:
# authors_pairwise_score_matrix = pd.read_csv('authors_pairwise_score_matrix.csv').drop('Unnamed: 0', axis=1)

In [275]:
# authors_pairwise_score_matrix.index = authors_pairwise_score_matrix.columns

In [390]:
S = authors_pairwise_score_matrix.values

In [391]:
S_enhance = S / S.sum(axis=1).max() + (np.eye(len(S)) -  S / S.sum(axis=1).max() * np.eye(len(S)))

In [392]:
d = 0.5
W = (1-d)*S_enhance + d*matrix_coop

In [394]:
W_frame = pd.DataFrame(data=W, index=authors_, columns=authors_)

In [395]:
def get_author_first_last_store(data: pd.DataFrame) -> tp.Tuple[tp.Dict, tp.Dict]:
    """Подсчет появления атрибутов автора в первый и последний момент времени.

    Args:
        data (DataFrame): данные авторов по их статьям и атрибутам.

    Returns:
        author_first_store, author_last_store (Tuple[Dict, Dict]): словари появления ключевых слов у авторов в первый и последний моменты времени.
    """
    author_first_store = {}
    author_last_store = {}
    list_types = ['keywords', 'topics']
    el_types = ['prim_topic', 'subfield', 'field', 'domain']
    for i, row in tqdm(data.iterrows(), total=data.shape[0]):
        author_name = row['Author name']
        if author_name not in author_first_store:
            author_first_store[author_name] = {}
            author_last_store[author_name] = {}
            
        for types in list_types:
            
            if types not in author_first_store[author_name]:
                author_first_store[author_name][types] = {} 
                author_last_store[author_name][types] = {} 
                
            for t in row[types]:
                if t in author_first_store[author_name][types]:
                    
                    if row['pub_date'] < author_first_store[author_name][types][t]:
                        author_first_store[author_name][types][t] = row['pub_date']
                        
                    if row['pub_date'] > author_last_store[author_name][types][t]:
                        author_last_store[author_name][types][t] = row['pub_date']
                else:
                    author_first_store[author_name][types].update({t: row['pub_date']})
                    author_last_store[author_name][types].update({t: row['pub_date']})
            
        for type in el_types:
            
            if type not in author_first_store[author_name]:
                author_first_store[author_name][type] = {}
                author_last_store[author_name][type] = {} 
                
            if row[type] in author_first_store[author_name][type]: 
                
                if row['pub_date'] < author_first_store[author_name][type][row[type]]:
                    author_first_store[author_name][type][row[type]] = row['pub_date']
    
                if row['pub_date'] > author_last_store[author_name][type][row[type]]:
                        author_last_store[author_name][type][row[type]] = row['pub_date']
            
            else:
                author_first_store[author_name][type].update({row[type]: row['pub_date']})
                author_last_store[author_name][type].update({row[type]: row['pub_date']})
    return author_first_store, author_last_store
            
                

In [396]:
author_first_store, author_last_store = get_author_first_last_store(data_notna)

100%|████████████████████████████████| 129433/129433 [00:10<00:00, 12134.45it/s]


In [397]:
first_pub = data_notna.groupby('Author name')['pub_date'].agg(min).to_dict()

In [398]:
def rele(author_name: str, attr: str, subattr: str, time_func_to=to_month) -> float:
    """Подсчет relevance.

    Args:
        author_name (str): автор, для которого считается reletion.
        attr (str): надатрибут.
        subattr (str): атрибуты, принадлежащие надатрибуту.
        time_func_to (Callable[Tuple[datetime, datetime], int]): подсчет разницы между временами, переведенными в года/месяца. 

    Returns:
        res (float): значение relevance для автора при определенном атрибуте и отрезке времени.
    """
    res = (time_func_to(author_first_store[author_name][attr][subattr], t_current) + 1) 
    delim = (time_func_to(first_pub[author_name], t_current) + 1)
    if delim == 0:
        print(f"RELE: Error in pub_date: {author_name, attr, subattr}")
        return 0
    res /= delim
    return res

In [399]:
def prox(author_name: str, attr: str, subattr: str, time_func_to=to_month) -> float:
    """Подсчет proximity.

    Args:
        author_name (str): автор, для которого считается reletion.
        attr (str): надатрибут.
        subattr (str): атрибуты, принадлежащие надатрибуту.
        time_func_to (Callable[Tuple[datetime, datetime], int]): подсчет разницы между временами, переведенными в года/месяца. 

    Returns:
        1 / delim (float): значение proximity для автора при определенном атрибуте и отрезке времени.
    """
    delim = (time_func_to(author_last_store[author_name][attr][subattr], t_current)+1)
    if delim == 0:
        print(f"PROX: Error in pub_date: {author_name, attr, subattr}")
        return 0
    return 1 / delim

In [400]:
def weight(psi: float, author_name: str, attr: str, subattr: str, time_func_to=to_month) -> float:
    """Подсчет веса вершины автора.

    Args:
        psi (float): параметр, отвечающий за приоритет между relevance и proximity. 
        author_name (str): автор, для которого считается reletion.
        attr (str): надатрибут.
        subattr (str): атрибуты, принадлежащие надатрибуту.
        time_func_to (Callable[Tuple[datetime, datetime], int]): подсчет разницы между временами, переведенными в года/месяца. 

    Returns:
        (float): линейная комбинация, отвечающая за вес вершины. 
    """
    # print(f'RELE {rele(author_name, attr, subattr)} PROX {prox(author_name, attr, subattr)}')
    return psi * rele(author_name, attr, subattr, time_func_to) + (1 - psi) * prox(author_name, attr, subattr, time_func_to)

In [401]:
def calculate_adj_matrix(authors: tp.List, all_authors: tp.List, authors_rel: pd.Series) -> np.array:
    """Подсчет матрицы смежности.

    Args:
        authors (List): список всех авторов с их ФИО.
        all_authors (List): список всех авторов с их Author ID.
        authors_rel (Series): данные, отражающие связь между статьей, датой публикации и авторами статьи.

    Returns:
        adjacency_matrix (Array): матрица смежности авторов по их публикациям.
    """
    adjacency_matrix = np.zeros((len(authors), len(authors)))
    for it, aus in tqdm(enumerate(authors_rel.values), total=len(authors_rel.values)):
        for a in aus:
            for b in aus:
                adjacency_matrix[all_authors.index(a), all_authors.index(b)] = 1
    return adjacency_matrix

In [402]:
adjacency_matrix = calculate_adj_matrix(authors_, all_authors, authors_rel)

100%|███████████████████████████████████| 76997/76997 [00:30<00:00, 2518.83it/s]


In [293]:
authors_ = list(map(lambda x: id2author[x], all_authors))

In [403]:
adjacency_matrix = pd.DataFrame(adjacency_matrix, index=authors_, columns=authors_)

In [404]:
def N_a_func(author: str, adj_matrix: pd.DataFrame) -> pd.Series:
    """Подсчет смежных к автору авторов по совместным публикациям.

    Args:
        author (str): автор, для которого строятся смежные авторы.
        adj_matrix (Array): матрица смежности авторов по их публикациям.

    Returns:
        (Series): смежные авторы с заданным автором.
    """
    return adj_matrix.loc[author][adj_matrix.loc[author] > 0]

In [450]:
def Q(author: str, attr: str, subattr: str, adj_matrix: np.array, psi: float, time_func_to=to_month) -> float:
    """Подсчет Quality. Чем выше качество узла, тем более он связан с другими узлами и имеет наибольший вес относительно атрибута.

    Args:
        author (str): автор, для которого считается Quality.
        attr (str): надатрибут.
        subattr (str): атрибуты, принадлежащие надатрибуту.
        adj_matrix (Array): матрица смежности авторов по их публикациям.
        psi (float): параметр, отвечающий за приоритет между relevance и proximity. 
        time_func_to (Callable[Tuple[datetime, datetime], int]): подсчет разницы между временами, переведенными в года/месяца.

    Returns:
        (float): значение Quality для автора при определенном атрибуте.
        
    """
    N_a = N_a_func(author, adj_matrix)
    numeric = 0
    for a_1 in N_a.index:
        for a_2 in N_a.index:
            if a_1 == a_2: continue
            numeric += adj_matrix.loc[a_1, a_2] == 1
    weight_list = []
    return numeric / adj_matrix.sum().sum() * weight(psi, author, attr, subattr, time_func_to)
    # 
    # * np.mean(weight_list)
        

In [451]:
def IA(a_i: str, a_j: str, adj_matrix: np.array) -> tp.List:
    """Влияние между двумя узлами. Задает расширение существующего сообщества новыми узлами.

    Args: 
        a_i (str): i-й автор.
        a_j (str): j-й автор.
        adj_matrix (Array): матрица смежности авторов по их публикациям.

    Returns:
        (List): список авторов, являющихся смежными для двух авторов.
    """
    return list(set(N_a_func(a_i, adj_matrix).index) & (set(N_a_func(a_j, adj_matrix).index)))

In [452]:
def D_IA(a_i: str, a_j: str, adj_matrix: np.array) -> float:
    """Плотность зоны влияния между узлами показывает, есть ли связи с узлами, которые потенциально могут находиться в рекомендуемом сообществе.

    Args: 
        a_i (str): i-й автор.
        a_j (str): j-й автор.
        adj_matrix (Array): матрица смежности авторов по их публикациям.

    Returns:
        (float): степень связи между узлами, которые являются смежными для двух авторов.
    """
    ia_node = IA(a_i, a_j, adj_matrix)
    if len(ia_node) <= 1:
        return 0
    numeric = 0
    for a_1 in ia_node:
        for a_2 in ia_node:
            if a_1 == a_2: continue
            numeric += adj_matrix.loc[a_1, a_2] == 1

    return numeric / (len(ia_node)) / (len(ia_node) - 1)

In [453]:
def NRS(a_i: str, a_j: str, attr: str, subattr: str, adj_matrix: np.array, S_matrix: np.array, psi: float, time_func_to=to_month) -> float:
    """Сила связи между двумя узлами.

    Args:
        a_i (str): i-й автор.
        a_j (str): j-й автор.
        attr (str): надатрибут.
        subattr (str): атрибуты, принадлежащие надатрибуту.
        adj_matrix (Array): матрица смежности авторов по их публикациям.
        S_matrix (Array): матрица схожести авторов по атрибутам.
        psi (float): параметр, отвечающий за приоритет между relevance и proximity. 
        time_func_to (Callable[Tuple[datetime, datetime], int]): подсчет разницы между временами, переведенными в года/месяца.
        
    Returns:
        (float): cила связи между двумя узлами.
    """
    return S_matrix.loc[a_i, a_j] * D_IA(a_i, a_j, adj_matrix) * sum([Q(a, attr, subattr, adj_matrix, psi, time_func_to) for a in IA(a_i, a_j, adj_matrix) if subattr in author_first_store[a][attr]])

In [454]:
def get_candidate(author, attr, subattr, adj_matrix, S_matrix, psi, time_func_to=to_month) -> str:
    """Выбор центрального узла-кандидата сообщества.

    Args:
        author (str): автор, для которого считается Quality.
        attr (str): надатрибут.
        subattr (str): атрибуты, принадлежащие надатрибуту.
        adj_matrix (Array): матрица смежности авторов по их публикациям.
        S_matrix (Array): матрица схожести авторов по атрибутам.
        psi (float): параметр, отвечающий за приоритет между relevance и proximity. 
        time_func_to (Callable[Tuple[datetime, datetime], int]): подсчет разницы между временами, переведенными в года/месяца.
        
    Returns:
        (str): центральный узел-кандидат.
    """
    N_a_author = N_a_func(author, adj_matrix)
    mass = Q(author, attr, subattr, adj_matrix, psi, time_func_to)
    max_nrs = 0
    candidate = author
    for v in N_a_author.index:
        if v == author or subattr not in author_first_store[v][attr]: continue
        if Q(v, attr, subattr, adj_matrix, psi, time_func_to) > mass:
            nrs_current = NRS(author, v, attr, subattr, adj_matrix, S_matrix, psi, time_func_to)
            if nrs_current > max_nrs:
                max_nrs = nrs_current
                candidate = v
    return candidate

In [455]:
def core_detecting(author: str, attr: str, subattr: str, adj_matrix: np.array, S_matrix: np.array, psi: float, time_func_to=to_month) -> str:
    """Выбор центрального узла.

    Args:
        author (str): автор, для которого считается Quality.
        attr (str): надатрибут.
        subattr (str): атрибуты, принадлежащие надатрибуту.
        adj_matrix (Array): матрица смежности авторов по их публикациям.
        S_matrix (Array): матрица схожести авторов по атрибутам.
        psi (float): параметр, отвечающий за приоритет между relevance и proximity. 
        time_func_to (Callable[Tuple[datetime, datetime], int]): подсчет разницы между временами, переведенными в года/месяца.
        
    Returns:
        (str): центральный узел целевого сообщества.
    """
    while True:
        candi = get_candidate(author, attr, subattr, adj_matrix, S_matrix, psi, time_func_to)
        if candi == author:
            break
        else:
            author = candi
    return author

In [456]:
author2id = {v: k for k, v in id2author.items()}

In [457]:
def N_a2(C: tp.List , adj_matrix: np.array) -> tp.List:
    """Подсчет соседей сообщества C.
    
    Args:
        C (List): авторы сообщества C.
        adj_matrix (Array): матрица смежности авторов по их публикациям.
    
    Returns:
        (List): соседи сообщества C.
    """
    conj_C = []
    for c in C:
        conj_C = conj_C + list(set(N_a_func(c, adj_matrix).index) - set(C))
    return list(set(conj_C))

In [458]:
def IA_2(v: str, C: tp.List, adj_matrix: np.array) -> tp.List:
    """Область влияния между сообществом C и его соседом v. 
    
    Args:
        v (str): автор, сосед сообщества C. 
        C (List): авторы сообщества C.
        adj_matrix (Array): матрица смежности авторов по их публикациям.
    
    Returns:
        (List): список авторов, являющихся смежными для сообщества и автора.
    """
    # if v not in N_a2(C):
    #     return list()
    # print('====')
    # print(C)
    # print(N_a_func(v).index)
    # print(list(set(C) & set(N_a_func(v).index)))
    return list(set(C) & set(N_a_func(v, adj_matrix).index))

In [459]:
def D_IA_2(v, C, adj_matrix):
    """Плотность зоны влияния между сообществом и узлом показывает, есть ли связи с узлами, которые потенциально могут находиться в рекомендуемом сообществе.

    Args: 
        a_i (str): i-й автор.
        a_j (str): j-й автор.
        adj_matrix (Array): матрица смежности авторов по их публикациям.

    Returns:
        (float): степень связи между узлами, которые являются смежными для двух авторов.
    """
    ia_node = IA_2(v, C, adj_matrix)
    numeric = 0
    for a_1 in ia_node:
        for a_2 in ia_node:
            numeric += adj_matrix.loc[a_1, a_2] == 1
    return numeric / (len(ia_node)) / (len(ia_node) - 1)

In [460]:
def CRS(v: str, C: tp.List, attr: str, subattr: str, adj_matrix: np.array, S_matrix: np.array, psi: float, time_func_to=to_month) -> float:
    """Сила отношения сообщества С c автором v.

    Args:
        v (str): автор, сосед сообщества C. 
        C (List): авторы сообщества C.
        attr (str): надатрибут.
        subattr (str): атрибуты, принадлежащие надатрибуту.
        adj_matrix (Array): матрица смежности авторов по их публикациям.
        S_matrix (Array): матрица схожести авторов по атрибутам.
        psi (float): параметр, отвечающий за приоритет между relevance и proximity. 
        time_func_to (Callable[Tuple[datetime, datetime], int]): подсчет разницы между временами, переведенными в года/месяца.

    Returns:
        (float): Сила отношения сообщества С c автором v.   
    """
    return np.mean([S_matrix.loc[v, c] for c in C]) * D_IA_2(v, C, adj_matrix) * sum([Q(a, attr, subattr, adj_matrix, psi, time_func_to) for a in IA_2(v, C, adj_matrix) if subattr in author_first_store[a][attr]]) 

In [461]:
attr = 'keywords'

In [462]:
def calculate_recommendation(core_member: str, adj_matrix: np.array, S_matrix: np.array, psi: float, time_func_to=to_month) -> tp.List:
    """Алгоритм расширения сообщества. Дополняет центральный узел узлами двумя способами: NRS и CRS. 

    Args: 
        core_member (str): автор, для которого строится рекомендация новых авторов и расширение сообщества.
        adj_matrix (Array): матрица смежности авторов по их публикациям.
        S_matrix (Array): матрица схожести авторов по атрибутам.
        psi (float): параметр, отвечающий за приоритет между relevance и proximity. 
        time_func_to (Callable[Tuple[datetime, datetime], int]): подсчет разницы между временами, переведенными в года/месяца.
        
    Returns:
        C_res (List): расширенное сообщество.
    """
    print('start')
    subattrs = list(author_first_store[core_member]['keywords'].keys())
    C_res = []
    for subattr in tqdm(subattrs):
        
        C = [core_detecting(core_member, attr, subattr, adj_matrix, S_matrix, psi, time_func_to)]
        N_C_init = N_a2(C, adj_matrix)
        iter = 1
        while True:
            com_size = len(C)
            # print("\n")
            # print(f"=====Итерация №{iter}=====")
            for v in N_C_init:
                # print(f"---проверка вершины v = {v}---")
                if subattr in author_first_store[v][attr]:
                    cand_v = get_candidate(v, attr, subattr, adj_matrix, S_matrix, psi, time_func_to)
                    if cand_v in C: # node relation strength
                        # print(f"кандидат вершины лежит в C, добавляем вершину: C = {C} \n v = {v}")
                        C = C + [v]
                    # print(C)
                    else: # community relation strength
                        # print(f"кандидат вершины не лежит в C, проверяем CRS")
                        # print(0)
                        crs_c = CRS(v, C, attr, subattr, adj_matrix, S_matrix, psi, time_func_to)
                        # print(crs_c)
                        # print(1)
                        C_conj = list(set(authors_) - set(C)) 
                        crs_conj_c = CRS(v, C_conj, attr, subattr, adj_matrix, S_matrix, psi, time_func_to)
                        # print(2) 
                        if crs_c > crs_conj_c:
                            # print(f"CRS кандидата v и сообщества С больше, чем CRS дополнения, добавляем вершину: C = {C} \n v = {v}")
                            C = C + [v]
                        # else:
                            # print(f"CRS дополнения больше, CRS сообщества для вершины v, v не добавлена в C")
                # else:
                    # print('У вершины v нет данного аттрибута, итерация пропущена.')
            if len(C) == com_size:
                # print("Итерация цикла не привела к увеличению C, процесс завершен.")
                break
            else:
               # print("Итерация цикла завершена, сообщество увеличилось, формируем новое дополнение к сообществу C") 
               N_C_init = N_a2(C, adj_matrix)
            iter += 1
            # print("\n")
        C_res.extend(C)
    return C_res

### Метрики

In [463]:
# Модулярность M(F)
def modularity_M(F: tp.List, weights: pd.DataFrame) -> float:
    """Метрика герметичности сообщества, сравнение силы внутреннего соединения и силы соединениядвух узлов в сообществе, когда оно выбрано случайным образом. Чем больше значение метрики, тем лучше.

    Args:
        F (List): cообщество авторов.
        weights (DataFrame): матрица сходности W авторов по временнным и атрибутным связям.

    Returns:
        Q_m (float): значение метрики.
    """
    # pairs = combinations(F, 2)
    w = W_frame.loc[F, F].sum().sum()
    Q_m = 0
    for a1 in tqdm(F):
        for a2 in F:
            if a1 == a2: continue
            Q_m += 1 / (2 * w) * (weights.loc[a1, a2] - np.dot(weights.loc[:, a2], weights.loc[a1, :]) / (2 * w))
    return Q_m
    # 1 / (2 * w) * np.sum([weights.loc[a1, a2] - np.dot(weights.loc[:, a2], weights.loc[a1, :]) / (2 * w) for a1, a2 in pairs])

In [464]:
def volume(F: tp.List, adj_matrix: pd.DataFrame) -> int:
    """Подсчет степеней вершин сообщества F.
    
    Args:
        F (List): cообщество авторов.
        adj_matrix (DataFrame):  матрица смежности авторов по их публикациям.

    Returns:
        vol (int): сумма степеней вершин сообщества F.
    """
    vol = 0
    for author in F:
        vol += adj_matrix.loc[author, :].sum()
    return vol

In [465]:
def P_cut(F: tp.List, authors: tp.List, weights: pd.DataFrame) -> float:
    """Отношение связности вершин внутри сообщества к вершинам вне сообщества.
    Args:
        F (List): cообщество авторов.
        authors (List): список всех авторов с их ФИО.
        weights (DataFrame): матрица сходности W авторов по временнным и атрибутным связям.

    Returns:
        pcut (int): значение метрики.
    """
    not_F = list(set(authors) - set(F))
    pcut = 0
    for a1 in F:
        sum_1 = 0
        sum_2 = 0
        for a2 in not_F:
            sum_1 += weights.loc[a1, a2] 
        for a2 in F:
            sum_2 += weights.loc[a1, a2] 
        pcut += sum_1 / sum_2
    return pcut 

In [466]:
def Cond_metric(F: tp.List, authors: tp.List, weights: pd.DataFrame, adj_matrix: pd.DataFrame) -> float:
    """Метрика плотности множества вершин в сообществе.
    
    Args:
        F (List): cообщество авторов.
        authors (List): список всех авторов с их ФИО.
        weights (DataFrame): матрица сходности W авторов по временнным и атрибутным связям.
        adj_matrix (DataFrame):  матрица смежности авторов по их публикациям.

    Returns:
        (float): значение метрики.
    """
    return P_cut(F, authors_, weights) / volume(F, adj_matrix)

### Графики

In [423]:
def make_graph_visualize(core_member: str, F: tp.List, adj_matrix: pd.DataFrame) -> None:
    graph_nodes_C = list(set(C_res)) 
    graph_nodes_N = list(set(N_a_func(core_member).index))
    
    adj_matrix_C = adj_matrix.loc[graph_nodes_C, graph_nodes_C]
    adj_matrix_N = adjacency_matrix.loc[graph_nodes_N, graph_nodes_N]
    
    authors_nodes = adj_matrix.columns
    
    authors2nodes = dict(zip(authors_nodes, list(range(len(adj_matrix)))))
    nodes2authors = {v: k for k, v in nodes2authors.items()}
    
    authors_nodes_recommended = list(set(graph_nodes_C) - set(graph_nodes_N))

    edges_C = []
    for idx_1 in range(len(adj_matrix_C.values)):
        for idx_2 in range(idx_1, len(adj_matrix_C.values)):
            if idx_1 == idx_2: continue
            if adj_matrix_C.values[idx_1, idx_2] == 1:
                edges.append((authors_nodes[idx_1], authors_nodes[idx_2], {'color': 'red', 'weight': 1}))
        
    edges_N = []
    for idx_1 in range(len(adj_matrix_N.values)):
        for idx_2 in range(idx_1, len(adj_matrix_N.values)):
            if idx_1 == idx_2: continue
            if adj_matrix_N.values[idx_1, idx_2] == 1:
                edges_N.append((authors_nodes[idx_1], authors_nodes[idx_2]))

    G = nx.Graph()
    G.add_nodes_from(list(zip(authors_nodes_recommended, [{"color": "red"}] * len(authors_nodes_recommended))))
    G.add_nodes_from(list(zip(graph_nodes_N,[{"color": "blue"}] * len(graph_nodes_N))))
    colored_dict = nx.get_node_attributes(G, 'color')

    edges_by_kw = []
    for i in range(len(graph_nodes_C)):
        for j in range(i, len(graph_nodes_C)):
            if i == j: continue
            weight = len(set(author_first_store[graph_nodes_C[i]]['keywords'].keys()) & set(author_first_store[graph_nodes_C[j]]['keywords'].keys()))
            if weight > 0:
                edges_by_kw.append((graph_nodes_C[i], graph_nodes_C[j], {'color': 'green', 'weight': weight*0.1}))

    edges_by_kw = [i for i in edges_by_kw if (i[0], i[1]) not in edges_C]

    G.add_edges_from(edges_by_kw + edges_C)

    color_edges = nx.get_edge_attributes(G, 'color')
    weight_edges = nx.get_edge_attributes(G, 'weight')

    fig = plt.figure(3, figsize=(16, 16), dpi=80)
    nx.draw_spring(G, with_labels=True, font_size=18, font_weight='bold', node_color=list(colored_dict.values()),
                   edge_color=list(color_edges.values()), width=list(weight_edges.values()))
    
    plt.savefig(f'graph_{core_member}.png')

# | set(N_a_func(core_member).index))
    

In [424]:
def modularity_graphic(F: tp.List, core_member: str, authors: tp.List, adj_matrix: np.array, weight: pd.DataFrame) -> None:
    N_a = list(N_a_func(core_member, adj_matrix).index)
    N_a_list = []
    # Q_m = modularity_Q(N_a_list, W_frame)
    not_N_a = set(F) - set(N_a)
    values_M = []
    
    for node in tqdm(N_a_list):
        N_a_list.append(node)
        values_M.append(modularity_M(N_a_list, weight))
        
    for node in not_N_a:
        N_a_list.append(node)
        values_M.append(modularity_M(N_a_list, weight))
        
    bad_nodes = np.random.choice(list(set(authors) - set(N_a_list)), 100, replace=False)
    
    for node in bad_nodes:
        N_a_list.append(node)
        values_M.append(modularity_M(N_a_list, weight))

    plt.plot(values_M, label='динамика M(x)')
    # plt.plot([23,23], [0.08, 0.25], color='red')
    plt.scatter([len(N_a) - 1], [values_M[len(N_a) - 1]], color='green', label='M(N(a))')
    plt.scatter([len(N_a_list) - 101], [values_M[len(N_a_list) - 101]], color='red', label='M(F)')
    plt.xlabel("Число авторов")
    plt.legend()
    plt.grid()
    plt.ylabel("M(x)")

In [425]:
def cond_graphic(F: tp.List, core_member: str, authors: tp.List, adj_matrix: np.array, weight: pd.DataFrame) -> None:
    N_a = list(N_a_func(core_member, adj_matrix).index)
    N_a_list = []
    # Q_m = modularity_Q(N_a_list, W_frame)
    not_N_a = set(F) - set(N_a)
    values_C = []
    
    for node in tqdm(N_a_list):
        N_a_list.append(node)
        values_C.append(Cond_metric(N_a_list, list(authors), weight, adjacency_matrix))
        
    for node in not_N_a:
        N_a_list.append(node)
        values_C.append(Cond_metric(N_a_list, list(authors), weight, adjacency_matrix))
        
    bad_nodes = np.random.choice(list(set(authors) - set(N_a_list)), 100, replace=False)
    
    for node in bad_nodes:
        N_a_list.append(node)
        values_C.append(Cond_metric(N_a_list, list(authors), weight, adjacency_matrix))

    plt.plot(values_C, label='динамика Cond(x)')
    # plt.plot([23,23], [0.08, 0.25], color='red')
    plt.scatter([len(N_a) - 1], [values_C[len(N_a) - 1]], color='green', label='Cond(N(a))')
    plt.scatter([len(N_a_list) - 101], [values_C[len(N_a_list) - 101]], color='red', label='Cond(F)')
    plt.xlabel("Число авторов")
    plt.legend()
    plt.grid()
    plt.ylabel("Cond(x)")